<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Define a simple search space
def create_model(hidden_dim):
    return nn.Sequential(
        nn.Linear(10, hidden_dim),
        nn.ReLU(),
        nn.Linear(hidden_dim, 1)
    )

def evaluate_model(model, data, target):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    model.train()
    for _ in range(10):  # Train for 10 epochs
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    return loss.item()

# Random search over different hidden dimensions
best_loss = float('inf')
best_model = None
data = torch.randn(100, 10)
target = torch.randn(100, 1)
for _ in range(10):  # Try 10 different architectures
    hidden_dim = random.randint(5, 100)
    model = create_model(hidden_dim)
    loss = evaluate_model(model, data, target)
    if loss < best_loss:
        best_loss = loss
        best_model = model
print(f'Best hidden dimension: {best_model[0].out_features}, Loss: {best_loss}')